# MHIST Dataset


## Load Data Part
Load the MHIST Dataset and Pre-processing the images.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tqdm
import os
import copy
import time
from torchvision.utils import save_image
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/content/drive/MyDrive/ECE1512/ProjectB')
import utils
import networks

In [3]:
from PIL import Image
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/ECE1512/ProjectB/mhist_dataset/images.zip", 'r')
zip_ref.extractall("./")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
def load_data (img_path, csv_path, split):
  annotations = pd.read_csv(csv_path)
  data = annotations[annotations['Partition'] == split].reset_index(drop=True)
  return data, img_path

def get_item(data, image_path, idx, transformer):
  image_full_name = os.path.join(image_path, data.iloc[idx]['Image Name'])
  x = Image.open(image_full_name)

  if transformer:
      x = transformer(x)

  label = data.iloc[idx]["Majority Vote Label"]
  if label == 'HP':
      y = 0 #torch.tensor([1, 0, 0, 0, 0])
  else:
      y = 1

  return x, y

def data_loader(train_data, test_data, train_image_path, test_image_path, batch_size):
  im_size = (128,128)
  channel = 3
  num_classes = 2
  mean = [0.485, 0.456, 0.406]
  std = [0.5, 0.5, 0.5]
  transformer = transforms.Compose([transforms.Resize(im_size),transforms.ToTensor(), transforms.Normalize(mean= mean, std = std)])

  data = {'images_train': [], 'labels_train': [],'images_test':[], 'labels_test':[]}
  class_names = torch.tensor([0, 1])

  ''' train data '''
  for idx in range(len(train_data)):
    x, y = get_item(train_data, train_image_path, idx, transformer)
    data['images_train'].append(torch.unsqueeze(x, dim=0))
    data['labels_train'].append(y)

  train_images = torch.cat(data['images_train'],dim = 0)
  train_labels = torch.tensor(data['labels_train'],dtype=torch.long)
  dst_train = utils.TensorDataset(train_images, train_labels)


  '''' test data '''
  for idx in range(len(test_data)):
    x, y = get_item(test_data, test_image_path, idx, transformer)
    data['images_test'].append(torch.unsqueeze(x, dim=0))
    data['labels_test'].append(y)
  test_images = torch.cat(data['images_test'],dim = 0)
  test_labels = torch.tensor(data['labels_test'],dtype=torch.long)
  dst_test = utils.TensorDataset(test_images, test_labels)

  testloader = torch.utils.data.DataLoader(dst_test, batch_size=batch_size, shuffle=False, num_workers = 0)

  return channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader

In [5]:
image_path = '/content/images'
csv_path = '/content/drive/MyDrive/ECE1512/ProjectB/mhist_dataset/annotations.csv'

train_data, train_image_path = load_data(image_path, csv_path, 'train')
test_data, test_image_path = load_data(image_path, csv_path, 'test')

mh_channel, mh_im_size, mh_num_classes, mh_class_names, mh_mean, mh_std, mh_dst_train, mh_dst_test, mh_testloader = data_loader(train_data, test_data, train_image_path,test_image_path, 128)

## Arugment Creating Part


In [6]:
NET = 'ConvNet'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = utils.get_network(NET,mh_channel,mh_num_classes).to(device)
class mh_argument():
  def __init__(self, device, ipc = 50):
    self.num_classes = mh_num_classes
    self.dataset = 'MHIST'
    self.channel = mh_channel
    self.im_size = mh_im_size
    self.mean = mh_mean
    self.std = mh_std
    self.ipc = ipc
    self.Iteration = 200 #K
    self.num_epochs = 2
    self.num_eval = 20
    self.num_op_step = 1
    self.lr_img = 0.1
    self.lr_net = 0.01
    self.epoch_eval_train = 20
    self.batch_train = 128
    self.outer_loop, self.inner_loop = utils.get_loops(self.ipc)
    self.dis_metric = 'ours'
    self.init = 'real'
    self.testloader = mh_testloader
    self.save_path = "MHISTresult"


    self.device = device
    self.dsa = False
    self.dsa_param = utils.ParamDiffAug()
    self.dc_aug_param = utils.get_daparam(self.dataset, model, NET, ipc = self.ipc)
    self.dsa_strategy = self.dc_aug_param['strategy']


## Task 1.2

### Part A

Calculated The FLOPs for model

In [ ]:
!pip install thop
from thop import profile
#Flops
input = torch.randn(1, mh_channel, 32, 32).to(device)
flops, params = profile(model, inputs=(input, ))
print(f'FLOPs: {flops}')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
FLOPs: 50771968.0


Use the real dataset to train the model

In [ ]:
NET = 'ConvNet'
num_epochs = 200 #Epoch times
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = utils.get_network(NET, mh_channel, mh_num_classes, mh_im_size).to(device)
args = mh_argument(device, 50)
lr = float(args.lr_net)
optimizer = optim.SGD(model.parameters(), lr=args.lr_net, momentum=0.9)
lr_schedule = [num_epochs//2+1]
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)

mh_train_loader = DataLoader(mh_dst_train, batch_size=128, shuffle=True)

for epoch in tqdm.tqdm(range(num_epochs)):
    train_loss, train_acc = utils.epoch('train', mh_train_loader, model, optimizer, criterion, args, aug=False)
    if epoch in lr_schedule:
      lr *= 0.1
      optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)

test_loss, test_acc = utils.epoch('test', args.testloader, model, optimizer, criterion, args, aug=True)
print(f'\nTrain Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

100%|██████████| 200/200 [15:22<00:00,  4.61s/it]



Train Loss: 0.0026, Test Loss: 1.2831, Train Acc: 1.0000, Test Acc: 0.7380


### Part BCD
The DataD_GradientM function below will created an synthetic dataset.
When args.init = 'real', the dataset will be initiallize with real dataset(d);
otherwide, the synthetic dataset will be created with Gaussian noise(c).
The synthetic dataset will be save as png for visualization(b).

**Reference**: Bo Zhao, Konda Reddy Mopuri, and Hakan Bilen. Dataset condensation with gradient matching. ICLR, 1(2):3, 2021. https://arxiv.org/abs/2006.05929.



In [7]:
"""
@inproceedings{
zhao2021DC,
title={Dataset Condensation with Gradient Matching},
author={Bo Zhao and Konda Reddy Mopuri and Hakan Bilen},
booktitle={International Conference on Learning Representations},
year={2021},
url={https://openreview.net/forum?id=mSAKhLYLSsl}
}
"""

def DataD_GradientM(NET, args):
  '''
  itype:
    Net -> str(): net type
    image_syn ->
    args -> obj: parameters
  rtype:
    data_save, visual_save, test_acc, train_acc -> list[],list[],list[],list[]
  '''
  if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)

  ''' orgainize image part '''
  indices_class = [[] for c in range(args.num_classes)]
  images_all = [torch.unsqueeze(mh_dst_train[i][0], dim=0) for i in range(len(mh_dst_train))]
  labels_all = [mh_dst_train[i][1] for i in range(len(mh_dst_train))]

  for i, lab in enumerate(labels_all):
    indices_class[lab].append(i)

  images_all = torch.cat(images_all, dim=0).to(args.device)
  labels_all = torch.tensor(labels_all, dtype=torch.long, device=args.device)

  for c in range(args.num_classes):
    print('class c = %d: %d real images'%(c, len(indices_class[c])))

  def get_images(c, n): # get random n images from class c
      idx_shuffle = np.random.permutation(indices_class[c])[:n]
      return images_all[idx_shuffle]

  ''' initialize the synthetic data '''
  image_syn = torch.randn(size=(args.num_classes*args.ipc, mh_channel, mh_im_size[0], mh_im_size[1]), dtype=torch.float, requires_grad=True, device=args.device)
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(args.num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1)

  if args.init == 'real':
    print('initialize synthetic data from random real images')
    for c in range(args.num_classes):
      image_syn.data[c*args.ipc:(c+1)*args.ipc] = get_images(c, args.ipc).detach().data
  else:
    print('initialize synthetic data from random noise')

  data_save = []   #final syn-dataset (data_save[0][0] = images; data_save[0][1] = labels)
  test_acc = []    #record test accuracy of every iteration
  train_acc = []   #record train accuracy of every iteration

  optimizer_img = torch.optim.SGD([image_syn, ], lr=args.lr_img, momentum=0.5)
  optimizer_img.zero_grad()
  criterion_img = nn.CrossEntropyLoss().to(args.device)


  for it in range(args.Iteration):
    print(f'Iteration {it}')

    ''' visualization of condensed images per class for MNIST datasets'''
    if it%10 == 0:
      save_name = os.path.join(args.save_path, args.init+'vis_%s_%dipc_iter%d.png'%( args.dataset, args.ipc, it))
      image_syn_vis = copy.deepcopy(image_syn.detach().cpu())
      for ch in range(args.channel):
        image_syn_vis[:, ch] = image_syn_vis[:, ch]  * args.std[ch] + args.mean[ch]
      image_syn_vis[image_syn_vis<0] = 0.0
      image_syn_vis[image_syn_vis>1] = 1.0
      save_image(image_syn_vis, save_name, nrow=args.ipc)

    ''' initial net '''
    net = utils.get_network(NET,args.channel,args.num_classes, args.im_size).to(args.device)
    net.train()
    net_parameters = list(net.parameters())
    optimizer_net = torch.optim.SGD(net.parameters(), lr=args.lr_net)
    optimizer_net.zero_grad()
    args.dc_aug_param = None

    for ol in tqdm.tqdm(range(args.outer_loop)): #outer loop

      ''' update synthesis data '''
      loss = torch.tensor(0.0).to(args.device)
      for c in range(args.num_classes): #class loop
        ''' Sample a minibatch pair BTc ∼ T and BSc ∼ S '''
        real_minibatch = get_images(c, args.batch_train)
        lab_real = torch.ones((real_minibatch.shape[0],), device=args.device, dtype=torch.long) * c
        synt_minibatch = image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, args.channel, args.im_size[0], args.im_size[1]))
        lab_synt = torch.ones((args.ipc,), device=args.device, dtype=torch.long) * c

        if args.dsa:
          seed = int(time.time() * 1000) % 100000
          img_real = utils.DiffAugment(img_real, args.dsa_strategy, seed=seed, param=args.dsa_param)
          img_syn = utils.DiffAugment(img_syn, args.dsa_strategy, seed=seed, param=args.dsa_param)


        ''' compute loss for both minibatch '''
        output_real = net(real_minibatch)
        loss_real = criterion_img(output_real, lab_real)
        gw_real = torch.autograd.grad(loss_real, net_parameters)
        gw_real = list((_.detach().clone() for _ in gw_real))

        output_synt = net(synt_minibatch)
        loss_synt = criterion_img(output_synt, lab_synt)
        gw_synt = torch.autograd.grad(loss_synt, net_parameters, create_graph=True)

        loss += utils.match_loss(gw_synt, gw_real, args)

      ''' update '''
      optimizer_img.zero_grad()
      loss.backward()
      optimizer_img.step()

      image_syn_train, label_syn_train = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach())
      dst_syn_train = utils.TensorDataset(image_syn_train, label_syn_train)
      trainloader = torch.utils.data.DataLoader(dst_syn_train, batch_size=args.batch_train, shuffle=True, num_workers=0)

      ''' inner outer '''
      for inner in range(args.inner_loop):
        utils.epoch('train', trainloader, net, optimizer_net, criterion_img, args, aug = True if args.dsa else False)

    ''' evaluate '''
    args.epoch_eval_train = 100
    net_eval = utils.get_network(NET, args.channel, args.num_classes, args.im_size).to(args.device) # get a random model
    image_syn_eval, label_syn_eval = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach())
    _, acc_train, acc_test = utils.evaluate_synset(it, net_eval, image_syn_eval, label_syn_eval, args.testloader, args)
    test_acc.append(acc_test)
    train_acc.append(acc_train)

    '''Save the synthetic data result'''
    if it == (args.Iteration-1):
      data_save.append([copy.deepcopy(image_syn.detach().cpu()), copy.deepcopy(label_syn.detach().cpu())])
      torch.save({'data': data_save}, os.path.join(args.save_path, args.init+'res_%s_%dipc.pt'%(args.dataset, args.ipc)))
  return data_save, test_acc, train_acc


### Part D

In [8]:
"""
Initialization Synthetic Dataset with Real Dataset
"""
NET = "ConvNet"
args = mh_argument(device, 50)
args.init = 'real'
start_time = time.time()
real_data_save, real_test_acc, real_train_acc = DataD_GradientM(NET, args)
print("time: ", time.time()-start_time)


class c = 0: 1545 real images
class c = 1: 630 real images
initialize synthetic data from random real images
Iteration 0


100%|██████████| 50/50 [03:03<00:00,  3.66s/it]


[2023-12-05 03:26:08] Evaluate_00: epoch = 0100 train time = 20 s train loss = 0.000039 train acc = 1.0000, test acc = 0.5241
Iteration 1


100%|██████████| 50/50 [02:57<00:00,  3.55s/it]


[2023-12-05 03:29:27] Evaluate_01: epoch = 0100 train time = 20 s train loss = 0.000136 train acc = 1.0000, test acc = 0.5415
Iteration 2


...

[2023-12-05 14:19:55] Evaluate_197: epoch = 0100 train time = 20 s train loss = 0.002212 train acc = 1.0000, test acc = 0.5435
Iteration 198


100%|██████████| 50/50 [02:57<00:00,  3.55s/it]


[2023-12-05 14:23:15] Evaluate_198: epoch = 0100 train time = 20 s train loss = 0.032163 train acc = 1.0000, test acc = 0.5200
Iteration 199


100%|██████████| 50/50 [02:57<00:00,  3.55s/it]


[2023-12-05 14:26:34] Evaluate_199: epoch = 0100 train time = 20 s train loss = 0.018323 train acc = 1.0000, test acc = 0.5619
time:  39832.006828546524


In [ ]:
"""
Initialliza Synthetice dataset with Gaussian Noise
"""
NET = "ConvNet"
args = mh_argument(device, 50)
args.init = 'noise'
start_time = time.time()
noise_data_save, noise_test_acc, noise_train_acc = DataD_GradientM(NET, args)
print("time:", time.time()-start_time)

class c = 0: 1545 real images
class c = 1: 630 real images
initialize synthetic data from random noise
Iteration 0


100%|██████████| 50/50 [00:11<00:00,  4.21it/s]


[2023-11-28 19:29:11] Evaluate_00: epoch = 0100 train time = 1 s train loss = 0.001948 train acc = 1.0000, test acc = 0.5384
Iteration 1


100%|██████████| 50/50 [00:11<00:00,  4.20it/s]


[2023-11-28 19:29:24] Evaluate_01: epoch = 0100 train time = 1 s train loss = 0.002119 train acc = 1.0000, test acc = 0.6080
Iteration 2


...

[2023-11-28 20:13:14] Evaluate_197: epoch = 0100 train time = 1 s train loss = 0.005633 train acc = 1.0000, test acc = 0.6694
Iteration 198


100%|██████████| 50/50 [00:11<00:00,  4.20it/s]


[2023-11-28 20:13:27] Evaluate_198: epoch = 0100 train time = 1 s train loss = 0.005626 train acc = 1.0000, test acc = 0.6571
Iteration 199


100%|██████████| 50/50 [00:11<00:00,  4.20it/s]


[2023-11-28 20:13:41] Evaluate_199: epoch = 0100 train time = 1 s train loss = 0.006469 train acc = 1.0000, test acc = 0.6592


### Part E
Use Synthetic Dataset to training model with the same paramter setting with part A, like epoch_eval_train = 200.

In [ ]:
Net = "ConvNet"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size).to(device)
print(model)
args = mh_argument(device, 50)
args.epoch_eval_train = 200

''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(real_data_save[0][0]), copy.deepcopy(real_data_save[0][1])
dst_syn = utils.TensorDataset(image_syn, label_syn)

start_time = time.time()
_, acc_train, acc_test = utils.evaluate_synset(1,model ,image_syn, label_syn, args.testloader, args)
print("test with synthetic dataset, accuracy = %.4f"%(acc_test))
print("time: ", time.time()-start_time)

ConvNet(
  (features): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(128, 128, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(128, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(128, 128, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Linear(in_features=2048, out_features=2, bias=True)
)
[2023-11-28 20:56:09] Evaluate_01: epoch = 0200 train time = 2 s train loss = 0.002266 train acc = 1.0000, test acc = 0.7226
test with synthetic dataset, accuracy = 0.7226
time:  2.960446834564209


## Task 1.3
In this part we trained different model with synthetic dataset

#### AlexNet

In [ ]:
Net = "AlexNet"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size).to(device)
print(model)
args = mh_argument(device, 50)
args.epoch_eval_train = 200

''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(real_data_save[0][0]), copy.deepcopy(real_data_save[0][1])
dst_syn = utils.TensorDataset(image_syn, label_syn)

_, acc_train, acc_test = utils.evaluate_synset(1,model ,image_syn, label_syn, args.testloader, args)
print("test with synthetic dataset, accuracy = %.4f"%(acc_test))

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(128, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3072, out_features=2, bias=True)
)
[2023-11-23 21:22:37] Evaluate_01: epoch = 0200 train time = 5 s train loss = 0.592060 train acc = 0.7800, test acc = 0.6

In [ ]:
#Noise Data testing
Net = "AlexNet"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size).to(device)
args = mh_argument(device, 50)
it_eval = args.num_eval
args.epoch_eval_train = 200


''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(noise_data_save[0][0]), copy.deepcopy(noise_data_save[0][1])
dst_syn = utils.TensorDataset(image_syn, label_syn)

_, acc_train, acc_test = utils.evaluate_synset(1, model, image_syn,label_syn, args.testloader, args)

[2023-11-23 21:24:12] Evaluate_01: epoch = 0200 train time = 5 s train loss = 0.036083 train acc = 1.0000, test acc = 0.4411


#### VGG11

In [ ]:
#Real Data Testing
Net = "VGG11"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size).to(device)
print(model)
args = mh_argument(device, 50)
args.epoch_eval_train = 200

''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(real_data_save[0][0]), copy.deepcopy(real_data_save[0][1])
dst_syn = utils.TensorDataset(image_syn, label_syn)

_, acc_train, acc_test = utils.evaluate_synset(1,model ,image_syn, label_syn, args.testloader, args)
print("test with synthetic dataset, accuracy = %.4f"%(acc_test))

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): GroupNorm(64, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(128, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(256, 256, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): GroupNorm(256, 256, eps=1e-05, affine=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16)

In [ ]:
#Noise Data testing
Net = "VGG11"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size).to(device)
args = mh_argument(device, 50)
it_eval = args.num_eval
args.epoch_eval_train = 200


''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(noise_data_save[0][0]), copy.deepcopy(noise_data_save[0][1])
dst_syn = utils.TensorDataset(image_syn, label_syn)
start_time = time.time()
_, acc_train, acc_test = utils.evaluate_synset(1, model, image_syn,label_syn, args.testloader, args)
vgg_time = time.time()-start_time
print("vgg_noisy_time = %.4f"%(vgg_time))

[2023-11-23 21:24:48] Evaluate_01: epoch = 0200 train time = 6 s train loss = 0.000169 train acc = 1.0000, test acc = 0.6551


#### AlexNet
use AlexNet To train real dataset

In [ ]:
NET = "AlexNet"
num_epochs = 200
lr = float(args.lr_net)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = utils.get_network(NET, mh_channel, mh_num_classes, mh_im_size).to(device)
args = mh_argument(device, 50)
optimizer = optim.SGD(model.parameters(), lr=args.lr_net, momentum=0.9)
lr_schedule = [num_epochs//2+1]
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)

mh_train_loader = DataLoader(mh_dst_train, batch_size=128, shuffle=True)
for epoch in tqdm.tqdm(range(num_epochs)):
    train_loss, train_acc = utils.epoch('train', mh_train_loader, model, optimizer, criterion, args, aug=False)
    if epoch in lr_schedule:
      lr *= 0.1
      optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)


test_loss, test_acc = utils.epoch('test', args.testloader, model, optimizer, criterion, args, aug=True)
print(f'\nTrain Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

100%|██████████| 200/200 [01:45<00:00,  1.90it/s]


Train Loss: 0.0002, Test Loss: 1.5989, Train Acc: 1.0000, Test Acc: 0.7441


#### MLP

In [ ]:
NET = "MLP"
num_epochs = 200
lr = float(args.lr_net)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = utils.get_network(NET, mh_channel, mh_num_classes, mh_im_size).to(device)
args = mh_argument(device, 50)
optimizer = optim.SGD(model.parameters(), lr=args.lr_net, momentum=0.9)
lr_schedule = [num_epochs//2+1]
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)

mh_train_loader = DataLoader(mh_dst_train, batch_size=128, shuffle=True)

for epoch in tqdm.tqdm(range(num_epochs)):
    train_loss, train_acc = utils.epoch('train', mh_train_loader, model, optimizer, criterion, args, aug=False)
    if epoch in lr_schedule:
      lr *= 0.1
      optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)

test_loss, test_acc = utils.epoch('test', args.testloader, model, optimizer, criterion, args, aug=True)
print(f'\nTrain Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

100%|██████████| 201/201 [00:12<00:00, 16.23it/s]


Train Loss: 0.0119, Test Loss: 1.6532, Train Acc: 0.9986, Test Acc: 0.7124


In [ ]:
#Real Syn Data Testing
Net = "MLP"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size)
print(model)
args = mh_argument(device, 50)
args.epoch_eval_train = 200

''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(real_data_save[0][0]), copy.deepcopy(real_data_save[0][1])
start_time = time.time()
_, acc_train, acc_test = utils.evaluate_synset(1,model ,image_syn, label_syn, args.testloader, args)
print("test with synthetic dataset, accuracy = %.4f"%(acc_test))
mlp_time = time.time() - start_time
print("mlp_time = %.4f"%(mlp_time))

MLP(
  (fc_1): Linear(in_features=3072, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=128, bias=True)
  (fc_3): Linear(in_features=128, out_features=2, bias=True)
)
[2023-11-23 22:19:32] Evaluate_01: epoch = 0200 train time = 0 s train loss = 0.001622 train acc = 1.0000, test acc = 0.7011
test with synthetic dataset, accuracy = 0.7011
mlp_time = 0.7561


#### LeNet


In [ ]:
NET = "LeNet"
num_epochs = 200
lr = float(args.lr_net)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = utils.get_network(NET, mh_channel, mh_num_classes, mh_im_size).to(device)
args = mh_argument(device, 50)
optimizer = optim.SGD(model.parameters(), lr=args.lr_net, momentum=0.9)
lr_schedule = [num_epochs//2+1]
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)

mh_train_loader = DataLoader(mh_dst_train, batch_size=128, shuffle=True)

for epoch in tqdm.tqdm(range(num_epochs)):
    train_loss, train_acc = utils.epoch('train', mh_train_loader, model, optimizer, criterion, args, aug=False)
    if epoch in lr_schedule:
      lr *= 0.1
      optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)

test_loss, test_acc = utils.epoch('test', args.testloader, model, optimizer, criterion, args, aug=True)
print(f'\nTrain Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

100%|██████████| 200/200 [00:14<00:00, 13.50it/s]


Train Loss: 0.0002, Test Loss: 2.2513, Train Acc: 1.0000, Test Acc: 0.7349


In [ ]:
#Real Syn Data Testing
Net = "LeNet"
model = utils.get_network(Net, mh_channel, mh_num_classes, mh_im_size)
print(model)
args = mh_argument(device, 50)
args.epoch_eval_train = 200

''' load Data Part'''
#syn dataset
image_syn, label_syn = copy.deepcopy(real_data_save[0][0]), copy.deepcopy(real_data_save[0][1])

start_time = time.time()
_, acc_train, acc_test = utils.evaluate_synset(1,model ,image_syn, label_syn, args.testloader, args)
print("test with synthetic dataset, accuracy = %.4f"%(acc_test))
letNet_time = time.time() - start_time
print("letNet_time: %.4f"%(letNet_time))

LeNet(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_1): Linear(in_features=400, out_features=120, bias=True)
  (fc_2): Linear(in_features=120, out_features=84, bias=True)
  (fc_3): Linear(in_features=84, out_features=2, bias=True)
)
[2023-11-23 22:19:39] Evaluate_01: epoch = 0200 train time = 0 s train loss = 0.304847 train acc = 0.9800, test acc = 0.6684
test with synthetic dataset, accuracy = 0.6684
letNet_time: 0.6883
